<a href="https://colab.research.google.com/github/Gabriel07201/Alura/blob/main/Data%20Science/Word%20Embedding/treino_novo_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
dados_treino = pd.read_csv(r'/content/drive/MyDrive/Cursos Alura/Word2Vec/treino.csv')
dados_teste = pd.read_csv(r'/content/drive/MyDrive/Cursos Alura/Word2Vec/teste.csv')

dados_treino.sample(5)

,title,text,date,category,subcategory,link
40427,As gralhas,É preciso ter cuidado com o uso dos pronomes. ...,2016-07-06,colunas,alvaro-costa-e-silva,http://www1.folha.uol.com.br/colunas/alvaro-co...
22430,Cuba prende manifestantes no Dia dos Direitos ...,Dezenas de dissidentes cubanos foram detidos n...,2015-11-12,mundo,NaN,http://www1.folha.uol.com.br/mundo/2015/12/171...
73677,Senado aprova indicação de Nikki Haley como em...,O Senado dos EUA aprovou nesta terça-feira (24...,2017-01-24,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/01/185...
15117,Litoral e interior de SP têm ao menos 15 casos...,O litoral e o interior do Estado de São Paulo ...,2015-08-12,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/12...
1718,"'Classe C está mais criteriosa no consumo', af...",A crise financeira mexe com a vida da classe C...,2015-11-18,mercado,NaN,http://www1.folha.uol.com.br/mercado/2015/11/1...


In [ ]:
dados_treino.shape

(90000, 6)

In [ ]:
!python -m spacy download pt_core_news_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-09 21:20:40.761711: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 68.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
import spacy
nlp = spacy.load('pt_core_news_sm')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
nlp

In [ ]:
texto = 'Curitiba é uma cidade daora'
nlp(texto)

Curitiba é uma cidade daora

In [ ]:
doc = nlp(texto)
type(doc)

spacy.tokens.doc.Doc

In [ ]:
doc[0]

Curitiba

In [ ]:
type(doc[0])

spacy.tokens.token.Token

In [ ]:
doc.ents

(Curitiba,)

In [ ]:
doc[0].is_stop

False

In [ ]:
doc[1].is_stop

True

In [ ]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino['title'])

In [ ]:
textos_para_tratamento

<generator object <genexpr> at 0x7f6741dfe7b0>

In [ ]:
def trata_textos(doc):
  tokens_validos = []
  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text)
  if len(tokens_validos) > 2:
    return ' '.join(tokens_validos)

# def trata_textos(doc):
#     tokens_validos = []
#     for token in doc:
#         e_valido = not token.is_stop and token.is_alpha
#         if e_valido:
#             tokens_validos.append(token.text)

#     if len(tokens_validos) > 2:
#         return  " ".join(tokens_validos)

In [ ]:
trata_textos(doc)

'Curitiba cidade daora'

In [ ]:
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size = 1000, n_process=-1)]

In [ ]:
len(textos_tratados)

90000

In [ ]:
titulos_tratados = pd.DataFrame({'titulo' : textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [ ]:
from gensim.models import Word2Vec

w2v = Word2Vec(sg=0, window=2, size=300, min_count=5, alpha=0.03, min_alpha=0.007)

In [ ]:
w2v

In [ ]:
print(len(titulos_tratados))
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

90000
84466


In [ ]:
lista_lista_tokens = [titulo.split(' ') for titulo in titulos_tratados['titulo']]

In [ ]:
w2v_model = Word2Vec(sg=0, window=2, size=300, min_count=5, alpha=0.03, min_alpha=0.007)

w2v_model.build_vocab(lista_lista_tokens)

In [ ]:
dir(w2v_model)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'ca

In [ ]:
w2v_model.corpus_count

84466

In [ ]:
w2v_model.train(lista_lista_tokens, total_examples=w2v_model.corpus_count, epochs=30)

(14584489, 16207260)

In [ ]:
w2v_model.wv.most_similar('google')

[('apple', 0.6053628921508789),
 ('facebook', 0.5188922882080078),
 ('amazon', 0.5058735609054565),
 ('uber', 0.48579204082489014),
 ('sony', 0.47261762619018555),
 ('news', 0.46975982189178467),
 ('waze', 0.4636436700820923),
 ('walmart', 0.452015221118927),
 ('airbnb', 0.4508505165576935),
 ('fbi', 0.446196049451828)]

In [ ]:
w2v_model.wv.most_similar('microsoft')

[('amazon', 0.6100612878799438),
 ('sony', 0.596057116985321),
 ('unilever', 0.588645339012146),
 ('walmart', 0.5409152507781982),
 ('tesla', 0.5333888530731201),
 ('braskem', 0.5332018136978149),
 ('viajante', 0.5252376794815063),
 ('canais', 0.5131992101669312),
 ('sky', 0.5123215317726135),
 ('spotify', 0.5096479654312134)]

In [ ]:
w2v_model.wv.most_similar('barcelona')

[('barça', 0.598699152469635),
 ('chelsea', 0.5761824250221252),
 ('juventus', 0.5532435178756714),
 ('madrid', 0.5464873313903809),
 ('bayern', 0.5450576543807983),
 ('psg', 0.5447651147842407),
 ('betis', 0.5429438352584839),
 ('united', 0.5345295667648315),
 ('leicester', 0.5270496606826782),
 ('figueirense', 0.5268184542655945)]

In [ ]:
w2v_model.wv.most_similar('messi')

[('suárez', 0.5846782922744751),
 ('cristiano', 0.5404399633407593),
 ('tevez', 0.521597146987915),
 ('cavani', 0.5035108327865601),
 ('barcelona', 0.5023453831672668),
 ('benzema', 0.4976503849029541),
 ('barça', 0.49031567573547363),
 ('neymar', 0.48339974880218506),
 ('psg', 0.48126164078712463),
 ('ronaldo', 0.4741111099720001)]

In [ ]:
w2v_model.wv.most_similar('gm')

[('embraer', 0.65871661901474),
 ('honda', 0.6426258683204651),
 ('chrysler', 0.6405819058418274),
 ('braskem', 0.6361047029495239),
 ('toyota', 0.6326950788497925),
 ('volks', 0.6246827840805054),
 ('volkswagen', 0.6017696261405945),
 ('fiat', 0.5915180444717407),
 ('sony', 0.5645104050636292),
 ('tesla', 0.5556093454360962)]

In [ ]:
w2v_model_sg = Word2Vec(sg=1, window=5, size=300, min_count=5, alpha=0.03, min_alpha=0.007)

w2v_model_sg.build_vocab(lista_lista_tokens)

w2v_model_sg.train(lista_lista_tokens, total_examples=w2v_model_sg.corpus_count, epochs=30)

(14584280, 16207260)

In [ ]:
w2v_model_sg.wv.most_similar('google')

[('apple', 0.4147097170352936),
 ('reguladores', 0.4036193788051605),
 ('android', 0.3920578956604004),
 ('anunciantes', 0.37193307280540466),
 ('waze', 0.36936795711517334),
 ('toshiba', 0.3606986701488495),
 ('chips', 0.3567999601364136),
 ('facebook', 0.34765830636024475),
 ('concorda', 0.3438401222229004),
 ('bmw', 0.341788649559021)]

In [ ]:
w2v_model_sg.wv.most_similar('gm')

[('metalúrgicos', 0.556614100933075),
 ('motors', 0.5194826126098633),
 ('honda', 0.5093482136726379),
 ('bmw', 0.4946836233139038),
 ('audi', 0.48926129937171936),
 ('montadora', 0.4704744815826416),
 ('cubatão', 0.466526061296463),
 ('airbag', 0.4623071253299713),
 ('compartilhamento', 0.4587538540363312),
 ('autoguiados', 0.45598140358924866)]

In [ ]:
w2v_model.wv.save_word2vec_format('/content/drive/MyDrive/Cursos Alura/Word2Vec/modelo_cbow.txt', binary=False)
w2v_model_sg.wv.save_word2vec_format('/content/drive/MyDrive/Cursos Alura/Word2Vec/modelo_skipgram.txt', binary=False)